In [1]:
!pip install medmnist
!pip install wandb
!pip install transformers
!pip installl pillow
!pip install evaluate
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.2 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha25

In [2]:
!wandb login


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mlabrie0208 (mlabrie0208-polytechnique-montr-al) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
import numpy as np
import torch
print(torch.cuda.is_available())
from torch.utils.data import Dataset
from transformers import (
    ViTHybridImageProcessor,
    ViTHybridForImageClassification,
    TrainingArguments,
    Trainer,
)
from medmnist import INFO
from medmnist.dataset import MedMNIST
import medmnist.dataset as mds
from PIL import Image
import evaluate



True


# Dataset

## Dataset class

In [4]:
class MedMNISTDataset(Dataset):
    def __init__(self, medmnist_data, feature_extractor):
        self.feature_extractor = feature_extractor
        # Use 'imgs' if available; otherwise, fallback to 'data'
        if hasattr(medmnist_data, 'imgs'):
            self.images = medmnist_data.imgs
        elif hasattr(medmnist_data, 'data'):
            self.images = medmnist_data.data
        else:
            raise ValueError("The MedMNIST dataset has neither 'imgs' nor 'data' attribute.")

        # Use 'labels' if available; otherwise, fallback to 'targets'
        if hasattr(medmnist_data, 'labels'):
            self.labels = medmnist_data.labels
        elif hasattr(medmnist_data, 'targets'):
            self.labels = medmnist_data.targets
        else:
            raise ValueError("The MedMNIST dataset has neither 'labels' nor 'targets' attribute.")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Retrieve the image (as a numpy array)
        image = self.images[idx]
        # Convert grayscale images to 3 channels if necessary
        if image.ndim == 2:
            image = np.stack([image] * 3, axis=-1)
        # Convert numpy array to PIL image
        image = Image.fromarray(image.astype('uint8'))
        # Retrieve the label and extract as an integer (avoiding DeprecationWarning)
        label = self.labels[idx]
        label = int(label.item()) if hasattr(label, "item") else int(label)
        # Process the image using the feature extractor (returns a dict with keys like 'pixel_values')
        inputs = self.feature_extractor(images=image, return_tensors="pt")
        # Remove the batch dimension from all tensor outputs
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["labels"] = torch.tensor(label)
        return inputs


## Loading Dataset

In [5]:
# Choose the MedMNIST dataset variant (e.g., 'pathmnist')
data_flag = 'pathmnist'
info = INFO[data_flag]

# Dynamically retrieve the dataset class (e.g., PathMNIST) from medmnist.dataset module.
DataClass = getattr(mds, info['python_class'])

# Download the train and validation splits
train_data = DataClass(split='train', download=True, ) # Change size
val_data = DataClass(split='val', download=True)

print(train_data.imgs[0].shape)



100%|██████████| 206M/206M [00:17<00:00, 11.6MB/s]


(28, 28, 3)


In [6]:
# Load the image processor from the pretrained checkpoint.
feature_extractor = ViTHybridImageProcessor.from_pretrained('google/vit-hybrid-base-bit-384')
print(feature_extractor.size)


# Set the number of labels based on the dataset information
num_labels = info['n_classes'] if 'n_classes' in info else len(info['label'])

# For the baseline, we load the model with ignore_mismatched_sizes so that the classifier head is re-initialized
baseline_model = ViTHybridForImageClassification.from_pretrained(
    'google/vit-hybrid-base-bit-384',
    num_labels=num_labels,
    ignore_mismatched_sizes=True,  # This ensures the classifier head matches our label count
)
baseline_model.config.id2label = {i: label for i, label in enumerate(info['label'])}
baseline_model.config.label2id = {label: i for i, label in enumerate(info['label'])}

# Also initialize the model for fine-tuning (it will be updated during training)
model = ViTHybridForImageClassification.from_pretrained(
    'google/vit-hybrid-base-bit-384',
    num_labels=num_labels,
    ignore_mismatched_sizes=True,
)
model.config.id2label = {i: label for i, label in enumerate(info['label'])}
model.config.label2id = {label: i for i, label in enumerate(info['label'])}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

{'shortest_edge': 384}


config.json:   0%|          | 0.00/71.9k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/396M [00:00<?, ?B/s]

Some weights of ViTHybridForImageClassification were not initialized from the model checkpoint at google/vit-hybrid-base-bit-384 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([9]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([9, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of ViTHybridForImageClassification were not initialized from the model checkpoint at google/vit-hybrid-base-bit-384 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([9]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([9, 768]) in the model instantiated
You should probably TRAIN this 

In [7]:
train_dataset = MedMNISTDataset(train_data, feature_extractor)
val_dataset = MedMNISTDataset(val_data, feature_extractor)


In [10]:
# Adjusted training arguments to use a smaller batch size and gradient accumulation.
training_args = TrainingArguments(
    output_dir="./vit-hybrid-medmnist",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=128,   # Reduced evaluation batch size as well
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    learning_rate=5e-5,
    report_to=["wandb"],  # Enable wandb logging
    run_name="vit-hybrid-medmnist-2e",
)

# Load the accuracy metric using the evaluate library.
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Training

In [13]:
torch.cuda.empty_cache()

baseline_trainer = Trainer(
    model=baseline_model,
    args=training_args,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Evaluate the baseline model on the validation set.
baseline_metrics = baseline_trainer.evaluate()
print("Baseline (pre-fine-tuning) metrics:", baseline_metrics)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mlabrie0208 (mlabrie0208-polytechnique-montr-al) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Baseline (pre-fine-tuning) metrics: {'eval_loss': 2.42923641204834, 'eval_model_preparation_time': 0.0061, 'eval_accuracy': 0.07287085165933627, 'eval_runtime': 268.8825, 'eval_samples_per_second': 37.206, 'eval_steps_per_second': 0.294}


In [11]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


# Create a Trainer for fine-tuning our model.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

torch.cuda.empty_cache()

# Start the fine-tuning process.
trainer.train()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mlabrie0208 (mlabrie0208-polytechnique-montr-al) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.035700,0.013804,0.995002


TrainOutput(global_step=1406, training_loss=0.06337648367305229, metrics={'train_runtime': 12192.5749, 'train_samples_per_second': 14.762, 'train_steps_per_second': 0.115, 'total_flos': 4.687189377451544e+19, 'train_loss': 0.06337648367305229, 'epoch': 1.9982225382154284})

In [12]:
# After training, evaluate the fine-tuned model on the validation set.
finetuned_metrics = trainer.evaluate()
print("Fine-tuned metrics:", finetuned_metrics)


Fine-tuned metrics: {'eval_loss': 0.013804325833916664, 'eval_accuracy': 0.9950019992003198, 'eval_runtime': 277.5537, 'eval_samples_per_second': 36.043, 'eval_steps_per_second': 0.285, 'epoch': 1.9982225382154284}
